In [1]:
import sys
import os
sys.path.append('../')
import raw_dr_ntu 
import process_dr_ntu
import pandas as pd
import json
from tqdm import tqdm

In [2]:
raw_dr_ntu_dir='./raw_dr_ntu'
raw_faculty_db = 'scse_profile'

process_faculty_db_dir ='./processed'
process_faculty_db = 'scse_profile'

In [3]:
scse_profiles = raw_dr_ntu.scrape_all_scse_profiles()
os.makedirs(raw_dr_ntu_dir, exist_ok=True)
output_file = os.path.join(raw_dr_ntu_dir, raw_faculty_db+'.csv')
scse_profiles.to_csv(output_file, index=False)
print(f'SCSE Profiles saved at {output_file}')

SCSE Profiles saved at ./raw_dr_ntu/scse_profile.csv


In [4]:
faculties = process_dr_ntu.generate_faculty_db(scse_profiles)
os.makedirs(process_faculty_db_dir, exist_ok=True)
output_file = os.path.join(process_faculty_db_dir, process_faculty_db+'.csv')
faculties.to_csv(output_file, index=False)
print(f'SCSE Profiles saved at {output_file}')

SCSE Profiles saved at ./processed/scse_profile.csv


In [5]:
for i, row in tqdm(faculties.iterrows()):
   profile = raw_dr_ntu.scrape_individual_profile(row['dr_ntu'], row['full_name'])
   raw_dr_ntu.save_individual_profile(profile, row['dr_ntu_id'], output_dir=raw_dr_ntu_dir)

In [7]:
faculties = pd.read_csv(os.path.join(process_faculty_db_dir, process_faculty_db+'.csv'))

for i, row in faculties.iterrows():
    with open(f"{raw_dr_ntu_dir}/{row['dr_ntu_id']}.json", 'r') as f:
        raw_dr_ntu_profile = json.load(f)

    google_scholar_id, google_scholar_url = process_dr_ntu.get_google_scholar(raw_dr_ntu_profile)

    faculties.at[i, 'google_scholar'] = google_scholar_url
    faculties.at[i, 'google_scholar_id'] = google_scholar_id

faculties.to_csv(os.path.join(process_faculty_db_dir, process_faculty_db+'.csv'), index=False)